In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [2]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Function to display images
def display_image(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Hide axes
    plt.show()

# Your remaining code goes here...


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Define the image file path
image_path = r"C:\Users\Victus\Desktop\RESEARCH\research\image1.png"  

# Check if the image exists
if os.path.exists(image_path):
    # Read the image
    im = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if im is not None:
        # Function to display images using matplotlib
        def display_image(image):
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.axis('off')  # Hide axes
            plt.show()

        # Display the image
        display_image(im)
    else:
        print("Error: Image could not be loaded. Please check the file format.")
else:
    print(f"Error: The file {image_path} does not exist.")


In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# Use Visualizer to draw the predictions on the image
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Display the image using the display_image function
display_image(out.get_image()[:, :, ::-1])  # Convert back to RGB for display

# Extract and print predictions
instances = outputs["instances"].to("cpu")
pred_classes = instances.pred_classes.numpy()  # Get predicted class labels
pred_scores = instances.scores.numpy()  # Get predicted scores

# Get class names from Metadata
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
class_names = metadata.get("thing_classes", [])

# Print predictions
print("Predictions:")
for cls, score in zip(pred_classes, pred_scores):
    class_name = class_names[cls] if cls < len(class_names) else "Unknown"
    print(f"Class: {class_name}, Score: {score:.2f}")


In [ ]:
def classify_anomalies(pred_classes, pred_scores, class_names, threshold=0.7):
    anomalies = []  # List to store detected anomalies

    for cls, score in zip(pred_classes, pred_scores):
        if score < threshold:  # Check if the score is below the threshold
            class_name = class_names[cls] if cls < len(class_names) else "Unknown"
            anomalies.append((class_name, score))

    return anomalies

# Classify anomalies in the detected instances
detected_anomalies = classify_anomalies(pred_classes, pred_scores, class_names)

# Print classified anomalies
print("Detected Anomalies:")
if detected_anomalies:
    for anomaly, score in detected_anomalies:
        print(f"Class: {anomaly}, Score: {score:.2f}")
else:
    print("No anomalies detected.")


In [ ]:
import zlib  # Import zlib for Deflate compression
import cv2  # Make sure cv2 is imported for image processing

def compress_image(image):
    """Compresses the given image using the Deflate algorithm."""
    if image is None or image.size == 0:  # Check if the image is empty
        raise ValueError("Input image is empty or not loaded correctly.")

    _, buffer = cv2.imencode('.png', image)  # Encode to PNG
    image_bytes = buffer.tobytes()  # Convert to byte array
    compressed_data = zlib.compress(image_bytes)  # Compress using zlib
    return compressed_data

def classify_anomalies(pred_classes, pred_scores, class_names, threshold=0.7):
    anomalies = []  # List to store detected anomalies

    for cls, score in zip(pred_classes, pred_scores):
        if score < threshold:  # Check if the score is below the threshold
            class_name = class_names[cls] if cls < len(class_names) else "Unknown"
            anomalies.append((class_name, score))

    return anomalies

# Example usage
# Assuming 'im' is the uploaded image and 'pred_classes', 'pred_scores', 'class_names' are defined
detected_anomalies = classify_anomalies(pred_classes, pred_scores, class_names)

# Print classified anomalies
print("Detected Anomalies:")
if detected_anomalies:
    for anomaly, score in detected_anomalies:
        print(f"Class: {anomaly}, Score: {score:.2f}")
    
    # Now perform compression on the uploaded image after printing anomalies
    compressed_image = compress_image(im)  # Use the uploaded image

    # Save compressed image data to a file or handle it as needed
    with open('compressed_image.deflate', 'wb') as f:
        f.write(compressed_image)
    print("Image compressed and saved as 'compressed_image.deflate'")
else:
    print("No anomalies detected; no compression performed.")


In [ ]:
import zlib  # Import zlib for Deflate compression
import cv2  # Ensure cv2 is imported for image processing
import numpy as np
import matplotlib.pyplot as plt
import os  # Import os for file operations

def eda_image(image):
    """Perform Exploratory Data Analysis on the given image."""
    if image is None or image.size == 0:  # Check if the image is empty
        raise ValueError("Input image is empty or not loaded correctly.")

    # Compute mean and standard deviation
    mean = np.mean(image)
    std_dev = np.std(image)

    # Plot histograms for each channel (assuming a color image)
    plt.figure(figsize=(12, 5))
    
    if len(image.shape) == 3:  # Color image
        colors = ('b', 'g', 'r')
        for i, color in enumerate(colors):
            hist = cv2.calcHist([image], [i], None, [256], [0, 256])
            plt.plot(hist, color=color)
            plt.xlim([0, 256])
    else:  # Grayscale image
        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        plt.plot(hist, color='gray')
        plt.xlim([0, 256])

    plt.title('Histogram of Color Channels')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.grid()
    plt.show()

    # Print EDA results
    print(f"EDA Results:\n"
          f"Mean Pixel Value: {mean:.2f}\n"
          f"Standard Deviation of Pixel Values: {std_dev:.2f}\n", flush=True)

def compress_image(image):
    """Compresses the given image using the Deflate algorithm and returns metrics."""
    if image is None or image.size == 0:  # Check if the image is empty
        raise ValueError("Input image is empty or not loaded correctly.")

    # Get original image properties
    original_size = image.nbytes  # Size in bytes
    image_format = 'PNG'  # Assuming we encode to PNG
    image_resolution = image.shape[1], image.shape[0]  # (width, height)

    # Start compression
    print("Starting compression...", flush=True)
    _, buffer = cv2.imencode('.png', image)  # Encode to PNG
    image_bytes = buffer.tobytes()  # Convert to byte array
    compressed_data = zlib.compress(image_bytes)  # Compress using zlib
    compressed_size = len(compressed_data)  # Size after compression

    # Calculate metrics
    compression_ratio = original_size / compressed_size
    bandwidth_saved = 100 * (1 - (compressed_size / original_size))
    
    # Estimate image quality (could be more complex)
    image_quality = 100 - (compressed_size / original_size * 100) if original_size > 0 else 100

    # Save the compressed image to a file
    output_file = 'compressed_image.deflate'
    with open(output_file, 'wb') as f:
        f.write(compressed_data)

    # Get the actual file size of the compressed image
    actual_compressed_size = os.path.getsize(output_file)

    # Print compression results
    print("Compression results (Deflate):", flush=True)
    print(f"Original file size: {original_size} bytes", flush=True)
    print(f"Compressed file size: {actual_compressed_size} bytes", flush=True)
    print(f"Compression ratio: {compression_ratio:.2f}", flush=True)
    print(f"Bandwidth saved: {bandwidth_saved:.2f}%", flush=True)
    print(f"Image Format: {image_format}", flush=True)
    print(f"Image Resolution: {image_resolution[0]}x{image_resolution[1]}", flush=True)
    print(f"Estimated Image Quality: {image_quality:.2f}", flush=True)

    return compressed_data

# Assuming 'im' is the uploaded image and 'pred_classes', 'pred_scores', 'class_names' are defined
detected_anomalies = classify_anomalies(pred_classes, pred_scores, class_names)

# Print classified anomalies
print("Detected Anomalies:", flush=True)
if detected_anomalies:
    for anomaly, score in detected_anomalies:
        print(f"Class: {anomaly}, Score: {score:.2f}", flush=True)

    # Check if the uploaded image is loaded correctly
    if im is None or im.size == 0:
        print("Error: The uploaded image is not loaded correctly.", flush=True)
    else:
        # Perform EDA on the uploaded image
        eda_image(im)  # Perform EDA on the uploaded image

        # Now perform compression on the uploaded image after EDA
        compressed_image = compress_image(im)  # Use the uploaded image

        # Save compressed image data to a file or handle it as needed
        print("Image compressed and saved as 'compressed_image.deflate'", flush=True)
else:
    print("No anomalies detected; no compression performed.", flush=True)
